In [1]:
#----------------
# variables
#----------------
img_height=64
img_width =384

n_train=300000  # needed training image number
n_test=1000   # needed testing image number
#--semi-fixed
data_dir       = "/home/apsisdev/ansary/DATASETS/synthdata_source" # resource
main_dir       = "/home/apsisdev/IMPORTANT/"  # saving 

In [2]:
#--------
# Resource fix
#--------

from coreLib.dataset import DataSet
from coreLib.utils import create_dir,LOG_INFO
ds=DataSet(data_dir)



 39%|███▉      | 78999/200840 [00:00<00:00, 789985.10it/s]

#LOG     :All paths found


  2%|▏         | 17996/1059977 [00:00<00:11, 92798.29it/s]

#LOG     :Processing Dictionary:/home/apsisdev/ansary/DATASETS/synthdata_source/bangla/dictionary.csv


100%|██████████| 10305/10305 [00:00<00:00, 1309290.64it/s]


...

#LOG     :bangla.graphemes


100%|██████████| 10305/10305 [00:00<00:00, 6912250.55it/s]


#LOG     :bangla.numbers
#LOG     :english.graphemes
#LOG     :english.numbers
#LOG     :common.symbols


100%|██████████| 3240/3240 [00:00<00:00, 6825487.17it/s]

#LOG     :bangla.fonts
#LOG     :english.fonts
#LOG     :common.background
#LOG     :common.noise.random
#LOG     :common.noise.sign


In [3]:
import os
#--------------
# Fixed structre
#--------------

save_dir=create_dir(main_dir,"BanglaSynthShort")
train_dir=create_dir(save_dir,"train")
test_dir=create_dir(save_dir,"test")

train_imgs_dir=create_dir(train_dir,"imgs")
train_label_txt=os.path.join(train_dir,"label.txt")
train_instances_txt=os.path.join(train_dir,"instances_train.txt")

test_imgs_dir=create_dir(test_dir,"imgs")
test_label_txt=os.path.join(test_dir,"label.txt")
test_instances_txt=os.path.join(test_dir,"instances_test.txt")

dict_txt=os.path.join(save_dir,"bangla_dict.txt")
dict_txt_grapheme=os.path.join(save_dir,"bangla_dict_grapheme.txt")

'''
# --> 
* bangalSynthShort
    banga_dict_next.txt
    * train
        * imgs
        * label.txt
        * instances_train.txt
    * test
        * imgs
        * label.txt
        * instances_test.txt-> label.txt
'''
print("Strcutre")

Strcutre


In [4]:
#---------------------
# data generation
#-----------------------
from coreLib.word import create_word
from coreLib.utils import stripPads
from coreLib.config import config
import cv2
import random
import matplotlib.pyplot as plt
import numpy as np
import os 
from tqdm.auto import tqdm

config.comp_dim=img_height

#---------------------------------------------------------------
# image utils
#---------------------------------------------------------------
def padImage(img,pad_loc,pad_dim):
    '''
        pads an image with white value
        args:
            img     :       the image to pad
            pad_loc :       (lr/tb) lr: left-right pad , tb=top_bottom pad
            pad_dim :       the dimension to pad upto 
    '''
    
    if pad_loc=="lr":
        # shape
        h,w=img.shape
        # pad widths
        left_pad_width =(pad_dim-w)//2
        # print(left_pad_width)
        right_pad_width=pad_dim-w-left_pad_width
        # pads
        left_pad =np.ones((h,left_pad_width))*255
        right_pad=np.ones((h,right_pad_width))*255
        # pad
        img =np.concatenate([left_pad,img,right_pad],axis=1)
    else:
        # shape
        h,w=img.shape
        # pad heights
        top_pad_height =(pad_dim-h)//2
        bot_pad_height=pad_dim-h-top_pad_height
        # pads
        top_pad =np.ones((top_pad_height,w))*255
        bot_pad=np.ones((bot_pad_height,w))*255
        # pad
        img =np.concatenate([top_pad,img,bot_pad],axis=0)
    return img.astype("uint8")    

def correctPadding(img,dim):
    '''
        corrects an image padding 
    '''
    img_height,img_width=dim
    # check for pad
    h,w=img.shape
    if w > img_width:
        # for larger width
        h_new= int(img_width* h/w) 
        img=cv2.resize(img,(img_width,h_new),fx=0,fy=0, interpolation = cv2.INTER_NEAREST)
        # pad
        img=padImage(img,pad_loc="tb",pad_dim=img_height) 
    elif w < img_width:
        # pad
        img=padImage(img,pad_loc="lr",pad_dim=img_width)
    # error avoid
    img=cv2.resize(img,(img_width,img_height),fx=0,fy=0, interpolation = cv2.INTER_NEAREST)
    return img 


In [5]:
'''
    charecter level dictionary: fixed data : 63
    
    ERROR CASE:
        GRAPHEME: Dynamic
'''
from coreLib.graphemeParser import GraphemeParser
GP=GraphemeParser()

dict_data=[]
dict_data_grapheme=[]
#---------------------
# train data
#-----------------------

# image--gen
#--------------------------------------------------
instances=[]
LABELS=[]



for i in tqdm(range(n_train)):
    try:
        '''
            White background : Black letters
        '''
        annotations=[]
        iden=3
        img,label,iden=create_word(iden,
                                   "bangla",
                                    random.choices(population=["handwritten","printed"],weights=[0.6, 0.4],k=1)[0],
                                    "grapheme",
                                    ds,
                                    use_dict=True)
        
        data=np.ones(img.shape)*255
        #{"char_text": "L", "char_box": [13.0, 14.0, 32.0, 12.0, 23.0, 36.0, 3.0, 38.0]}
        com=""
        for k,v in label.items():
            if v!=" ":
                #label
                com+=v
                # image creation
                data[img==k]=0
                # annotation
                idx = np.where(img==k)
                y_min,y_max,x_min,x_max = np.min(idx[0]), np.max(idx[0]), np.min(idx[1]), np.max(idx[1])
                annon={}
                annon["char_text"]=v 
                annon["char_box"]=[float(i) for i in [x_min,y_min,x_max,y_min,x_max,y_max,x_min,y_max]]  
                annotations.append(annon)

        # img save
        data=stripPads(data,255)
        data=correctPadding(data,dim=(img_height,img_width))
        cv2.imwrite(os.path.join(train_imgs_dir,f"{i}.png"),data)

        # instaces
        instances.append({"file_name": f"{i}.png", "annotations":annotations,"text":com})
        # label
        LABELS.append(f"{i}.png {com}")
        
        
        # dictionary formation
        # char
        for c in com:
            if c not in dict_data:
                dict_data.append(c)
        # grapheme
        coms=GP.word2grapheme(com)
        for c in coms:
            if c not in dict_data_grapheme:
                dict_data_grapheme.append(c)
    except Exception as e:
        print(e)
        
# instances creation
#------------------------------------------------------------------------
inst_text=[str(i) for i in tqdm(instances)] # text-conv
with open(train_instances_txt,"w") as f:
    for line in tqdm(inst_text):
        line=line.replace("'",'"')
        f.write(f"{line}\n")
# label creation
#-----------------------------------------------------
with open(train_label_txt,"w") as f:
    for line in tqdm(LABELS):
        f.write(f"{line}\n")


  0%|          | 0/300000 [00:00<?, ?it/s]

unsupported operand type(s) for +=: 'NoneType' and 'str'
empty range for randrange() (0, 0, 0)
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
empty range for randrange() (0, 0, 0)
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
empty range for randrange() (0, 0, 0)
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for

  0%|          | 0/299975 [00:00<?, ?it/s]

  0%|          | 0/299975 [00:00<?, ?it/s]

  0%|          | 0/299975 [00:00<?, ?it/s]

In [6]:
#---------------------
# test data
#-----------------------
LABELS=[]
# image--gen
#--------------------------------------------------
for i in tqdm(range(n_train,n_train+n_test)):
    try:
        '''
            White background : Black letters
        '''
        annotations=[]
        iden=3
        img,label,iden=create_word(iden,
                                   "bangla",
                                    random.choices(population=["handwritten","printed"],weights=[0.6, 0.4],k=1)[0],
                                    "grapheme",
                                    ds,
                                    use_dict=True)
        
        data=np.ones(img.shape)*255
        #{"char_text": "L", "char_box": [13.0, 14.0, 32.0, 12.0, 23.0, 36.0, 3.0, 38.0]}
        com=""
        for k,v in label.items():
            if v!=" ":
                #label
                com+=v
                # image creation
                data[img==k]=0
                
        # img save
        data=stripPads(data,255)
        data=correctPadding(data,dim=(img_height,img_width))
        cv2.imwrite(os.path.join(test_imgs_dir,f"{i}.png"),data)

        # label
        LABELS.append(f"{i}.png {com}")
        
        
        # dictionary formation
        # char
        for c in com:
            if c not in dict_data:
                dict_data.append(c)
        # grapheme
        coms=GP.word2grapheme(com)
        for c in coms:
            if c not in dict_data_grapheme:
                dict_data_grapheme.append(c)
    except Exception as e:
        print(e)
        
# instances creation
#------------------------------------------------------------------------
with open(test_instances_txt,"w") as f:
    for line in tqdm(LABELS):
        f.write(f"{line}\n")
# label creation
#-----------------------------------------------------
with open(test_label_txt,"w") as f:
    for line in tqdm(LABELS):
        f.write(f"{line}\n")

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
#---------------------
# dict
#-----------------------
dict_data=sorted(dict_data)
dict_data_grapheme=sorted(dict_data_grapheme)
# dicts
with open(dict_txt,"w") as f:
    for c in dict_data:
        f.write(f"{c}\n")

with open(dict_txt_grapheme,"w") as f:
    for c in dict_data_grapheme:
        f.write(f"{c}\n")


In [8]:
ls

 BN-HTRd_Analysis.ipynb         d.png                      PoC/
 coreLib/                       memoCreation_debug.ipynb   README.md
 debug.ipynb                    memoLib/                   requirements.txt
'detection_data (copy).ipynb'   memo.png                   store.py
 detection_data.ipynb           mobassir_segOCR.ipynb      UPDATE/


In [9]:
pwd

'/home/apsisdev/ansary/CODES/synthdata'

In [10]:
cd "/home/apsisdev/IMPORTANT/BanglaSynthShort"

/home/apsisdev/IMPORTANT/BanglaSynthShort


In [11]:
ls

bangla_dict_grapheme.txt  bangla_dict.txt  test/  train/


In [12]:
ls "/home/apsisdev/IMPORTANT/BanglaSynthShort/train"

imgs/  instances_train.txt  label.txt


In [13]:
ls "/home/apsisdev/IMPORTANT/BanglaSynthShort/test"

imgs/  instances_test.txt  label.txt


In [14]:
ls "/home/apsisdev/IMPORTANT/BanglaSynthShort/test/imgs"

300000.png  300185.png  300370.png  300555.png  300740.png  300925.png
300001.png  300186.png  300371.png  300556.png  300741.png  300926.png
300002.png  300187.png  300372.png  300557.png  300742.png  300927.png
300003.png  300188.png  300373.png  300558.png  300743.png  300928.png
300004.png  300189.png  300374.png  300559.png  300744.png  300929.png
300005.png  300190.png  300375.png  300560.png  300745.png  300930.png
300006.png  300191.png  300376.png  300561.png  300746.png  300931.png
300007.png  300192.png  300377.png  300562.png  300747.png  300932.png
300008.png  300193.png  300378.png  300563.png  300748.png  300933.png
300009.png  300194.png  300379.png  300564.png  300749.png  300934.png
300010.png  300195.png  300380.png  300565.png  300750.png  300935.png
300011.png  300196.png  300381.png  300566.png  300751.png  300936.png
300012.png  300197.png  300382.png  300567.png  300752.png  300937.png
300013.png  300198.png  300383.png  300568.png  300753.png  300938.png
300014